<a href="https://colab.research.google.com/github/higero0324/portfolio/blob/main/%E8%AA%B2%E9%A1%8C04_20_TS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **5*5グリッドダンジョン**

## **概要**

このゲームは5*5で構成されたダンジョンの端にあるスタートからから、対角の端にあるゴールまでたどり着いたらゲームクリアとなるゲームです。その中で、移動中にエネミーと接敵し、倒せれば先に進めます。エネミーの場所はわからないので、とりあえず進んでみて、エネミーと遭遇すれば戦闘と逃げるを選べます。運が良ければ接敵しない場合もあります。

## **操作方法**

操作は基本的に、書いてあるガイド従えばOKです。上下左右などは、しっかりと変換してから入力するようおねがいします。一応[up/down/left/right]などの英語も対応してます。升目は、0~4までのごマスが移動可能なのでご注意をお願いします。

## **制作者のコメント**

とても難しいプログラムで有識者やGPTに教えてもらいながら頑張りました。重すぎて途中作りながらプログラムを編集することが困難となり、何度もやめようと思いましたが、何とかやり切りました。あと、一応英語対応しもしましたが、いらなかった気もしました。

## **作成時間**

作成時間：**約70時間**

#**プログラム**

In [ ]:
%reset -f

import random
import matplotlib.pyplot as plt
from IPython.display import clear_output

class MazeGame:
    def __init__(self):
        self.maze_size = 5
        self.player_position = [0, 0]
        self.exit_position = [self.maze_size-1, self.maze_size-1]
        self.player_hp = 100
        self.player_level = 1
        self.player_attack = 10
        self.inventory = []
        self.monsters = self.generate_monsters()
        self.items = self.generate_items()
        self.game_over = False

    def generate_empty_positions(self):
        """プレイヤーと出口がない位置を取得"""
        empty_positions = [(x, y) for x in range(self.maze_size) for y in range(self.maze_size)
                           if [x, y] != self.player_position and [x, y] != self.exit_position]
        return empty_positions

    def generate_monsters(self):
        monsters = {}
        available_positions = self.generate_empty_positions()
        for _ in range(3):
            position = random.choice(available_positions)
            available_positions.remove(position)
            monsters[position] = {"hp": 50, "attack": 5}
        return monsters

    def generate_items(self):
        items = {}
        available_positions = self.generate_empty_positions()
        for _ in range(2):
            position = random.choice(available_positions)
            available_positions.remove(position)
            items[position] = random.choice(["health_potion", "attack_boost"])
        return items

    def display_status(self):
        print(f"プレイヤーのHP: {self.player_hp}")
        print(f"プレイヤーのレベル: {self.player_level}")
        print(f"プレイヤーの攻撃力: {self.player_attack}")
        print(f"持ち物: {self.inventory}")
        print(f"現在の位置: {self.player_position}\n")

    def move_player(self, direction):
        if direction == "up" and self.player_position[0] > 0:
            self.player_position[0] -= 1
        elif direction == "down" and self.player_position[0] < self.maze_size - 1:
            self.player_position[0] += 1
        elif direction == "left" and self.player_position[1] > 0:
            self.player_position[1] -= 1
        elif direction == "right" and self.player_position[1] < self.maze_size - 1:
            self.player_position[1] += 1
        else:
            print("その方向には進めません！")

        self.plot_position()
        self.check_position()

    def plot_position(self):
        clear_output(wait=True)
        fig, ax = plt.subplots()
        ax.set_xlim(-1, self.maze_size)
        ax.set_ylim(-1, self.maze_size)
        ax.grid(True)
        plt.plot(self.player_position[1], self.maze_size - 1 - self.player_position[0], "ro")
        plt.plot(self.exit_position[1], self.maze_size - 1 - self.exit_position[0], "go")
        plt.show()

    def check_position(self):
        if tuple(self.player_position) in self.monsters:
            self.battle(self.monsters[tuple(self.player_position)])
        elif tuple(self.player_position) in self.items:
            self.collect_item(self.items[tuple(self.player_position)])
        elif self.player_position == self.exit_position:
            print("出口にたどり着きました！あなたの勝ちです！")
            self.game_over = True

    def battle(self, monster):
        print("野生のモンスターが現れた！")
        while monster["hp"] > 0 and self.player_hp > 0:
            print(f"モンスターのHP: {monster['hp']}")
            action = input("[攻撃] または [逃げる] を選んでください: ").strip().lower()
            if action == "attack" or action == "攻撃":
                monster["hp"] -= self.player_attack
                if monster["hp"] <= 0:
                    print("モンスターを倒しました！")
                    self.player_level += 1
                    self.player_attack += 5
                    del self.monsters[tuple(self.player_position)]
                else:
                    self.player_hp -= monster["attack"]
                    print(f"モンスターが攻撃してきた！あなたのHPは {self.player_hp} です")
                    if self.player_hp <= 0:
                        print("モンスターに敗北しました...")
                        self.game_over = True
            elif action == "run" or action == "逃げる":
                print("無事に逃げ出しました！")
                break
            else:
                print("無効な選択です。")

    def collect_item(self, item):
        if item == "health_potion":
            item_jp = "回復ポーション"
        elif item == "attack_boost":
            item_jp = "攻撃力アップ"

        print(f"{item_jp}を見つけました！")
        self.inventory.append(item_jp)
        if item == "health_potion":
            self.player_hp += 20
            print("回復ポーションを使ってHPが20回復しました！")
        elif item == "attack_boost":
            self.player_attack += 10
            print("攻撃力アップを使って攻撃力が10増加しました！")
        del self.items[tuple(self.player_position)]

    def play(self):
        print("迷路ゲームへようこそ！")
        while not self.game_over:
            self.display_status()
            direction = input("[上/下/左/右] に移動してください: ").strip().lower()
            if direction == "上":
                direction = "up"
            elif direction == "下":
                direction = "down"
            elif direction == "左":
                direction = "left"
            elif direction == "右":
                direction = "right"
            self.move_player(direction)
        print("ゲームオーバー。")


game = MazeGame()
game.play()